![header](../figures/logos_partenaires._cmems_se.jpg)

# Demo SWOT Swath Filtering: Variational Filter (Gomez et al., 2018)

<div style="text-align: right"><i> 2022-11-07 DEMO GOMEZ FILTER </i></div>

***
**Authors:**  CLS & Datlas <br>
**Copyright:** 2022 CLS & Datlas <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1><center>Demo Gomez et al. filter</center></h1></div>

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***


<div class="alert alert-danger" role="alert">

<h3>Learning outcomes</h3>

At the end of this notebook you will know:
<ul>
  <li>How you can the Gomez et al. (2018) variational filter to SWOT Level3 SSH unfiltered products </li>
</ul>
    
</div>



In [ ]:
import os
import glob
import sys
import numpy as np
sys.path.append('..')
from src.swot import *
from src.plot import *
import warnings
warnings.filterwarnings("ignore")

<div class="alert alert-info" role="alert">

<h2>1. Input files to analyse</h2>

</div>

## Data challenge inputs

In [ ]:
# Check if inputs have already been downloaded otherwise do
if not os.path.isdir('../dc_inputs'):
    # Get pseuso-observation
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/catalog/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2022a_SWOT_karin_error_filtering/dc_inputs.tar.gz
    !tar -xvf dc_inputs.tar.gz --directory ../
    !rm -f dc_inputs.tar.gz

In [ ]:
list_of_file = sorted(glob.glob('../dc_inputs/*.nc'))
len(list_of_file)

## Outputs folder

In [ ]:
output_dir = '../results/demo_gomez_filtering'
os.system(f'mkdir -p {output_dir}')

## Filter function

In [ ]:
def run_filter(filename):
    swt = SwotTrack(filename) 
    swt.fill_nadir_gap('ssh_karin')
    swt.apply_var_filter('ssh_karin', 'ssh_karin_filt')
    swt.empty_nadir_gap('ssh_karin_filt')
    swt.empty_nadir_gap('ssh_karin')
    output_filename = f'{output_dir}/{os.path.basename(filename)}'
    swt.to_netcdf('all', output_filename)

# RUN FILTER

### PARALLEL VERSION

In [ ]:
from distributed import Client
import dask_jobqueue
import dask.bag

cluster = dask_jobqueue.SGECluster(queue='dask', cores=1, memory='2GB', resource_spec='mem_total=2G', processes=1, local_directory='$TMPDIR')
cluster.scale(200)

client = Client(cluster)
client.upload_file('../src/swot.py')
client

In [ ]:
bag = dask.bag.from_sequence(list_of_file)

In [ ]:
res = bag.map(run_filter).compute()

In [ ]:
client.close()
cluster.close()

### SERIAL VERSION

In [ ]:
%%time
for filename in list_of_file:
    run_filter(filename)

In [ ]:
plot_demo_pass(list_of_file[3], f'{output_dir}/{os.path.basename(list_of_file[3])}')